In [3]:
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import geopandas as gpd
import pandas as pd
import numpy as np
import nexus_tool.weap_tools as wp
import re
from functools import reduce
import os
from shutil import copyfile

In [40]:
def extract_weap(results, spacial_data, points, melt_on, regex):
    df = pd.DataFrame()
    for key, row in spacial_data.groupby(points):
        df_temp = results.filter(regex=regex.format(key)).copy()
        df_temp.columns = [i.split('\\')[-1] for i in df_temp.columns]
        df_temp[points] = key
        for i in melt_on:
            df_temp[i] = results[i]
        melt_vars = melt_on.copy()
        melt_vars.append(points)
        df = df.append(df_temp.melt(id_vars=melt_vars), sort=False)
    return df.reset_index(drop=True)

In [41]:
def extract(s, left, rigth):
    mo = re.search(f'{left}(.*){rigth}', s)
    if mo:
        return mo.group(1)
    return ''

In [42]:
def get_demand_data(sheet_names, data, spatial_data, variable, regex):
    merged_data = pd.DataFrame()
    for sheet_name, value in sheet_names.items():
        data_temp = data.parse(sheet_name, skiprows=5)
        data_temp.rename(columns={'$Columns = Year': 'Year', ' Timestep': 'Month'}, inplace=True)
        data_temp.columns = data_temp.columns.str.replace('"', '')
        merged_data_temp = extract_weap(data_temp, spatial_data, variable, ['Year','Month'], regex)
        merged_data_temp['units'] = [extract(s, '\[','\]') for s in merged_data_temp.variable]
        merged_data_temp['variable'] = [i.split('[')[0] for i in merged_data_temp.variable]
        merged_data_temp['type'] = value
        merged_data_temp.loc[merged_data_temp['type']=='variable', 'type'] = merged_data_temp.loc[merged_data_temp['type']=='variable', 'variable']

        merged_data = merged_data.append(merged_data_temp, sort=False)
        
    return merged_data

In [43]:
def enumerate_segments():
    x = len(pl_flow.Year.unique())*12
    pl_flow['n'] = 0
    for key, group in pl_flow.groupby('pipeline'):
        n = [[i]*x for i in range(int(group.shape[0]/x))]
        n = reduce(lambda a, b: a + b, n)
        pl_flow.loc[pl_flow.pipeline==key,'n'] = n

In [44]:
def get_elevation_delta():  
    pl_flow['elevation_delta'] = 0
    for pipeline in pl_flow.pipeline.unique():
        for n in range(pl_flow.loc[pl_flow.pipeline==pipeline, 'n'].max()):
            elevation_1 = pl_flow.loc[(pl_flow.pipeline==pipeline) & (pl_flow.n==n), 'elevation']
            elevation_2 = pl_flow.loc[(pl_flow.pipeline==pipeline) & (pl_flow.n==(n+1)), 'elevation']
            pl_flow.loc[(pl_flow.pipeline==pipeline) & (pl_flow.n==n), 'elevation_delta'] = np.array(elevation_2) - np.array(elevation_1)

In [45]:
files_dict = {'Reference': {'Eto trend': ['DataExchange - Reference.xlsm'], 
                            'Without Eto trend': ['DataExchange - Reference - No ETo Trend.xlsm']},
              'Improve AG eff': {'Eto trend': ['DataExchange - Improve AG eff by 10percent.xlsm',
                                               'DataExchange - Improve AG eff by 20percent.xlsm'], 
                                 'Without Eto trend': ['DataExchange - Improve AG eff by 10percent - No ETo Trend.xlsm',
                                                       'DataExchange - Improve AG eff by 20percent - No ETo Trend.xlsm']},
              'New Resources': {'Eto trend': ['DataExchange - New Resources.xlsm'], 
                                'Without Eto trend': ['DataExchange - New Resources - No ETo Trend.xlsm']},
              'Reduce NRW': {'Eto trend': ['DataExchange - Reduce NRW to 40 percent.xlsm', 
                                           'DataExchange - Reduce NRW to 20 percent.xlsm'], 
                             'Without Eto trend': ['DataExchange - Reduce NRW to 40 percent - No ETo Trend.xlsm',
                                                   'DataExchange - Reduce NRW to 20 percent - No ETo Trend.xlsm',]}}

def create_folder(path):
    if not os.path.exists(path):
        os.mkdir(path)

raw_data = os.path.join('Data/WEAP Results', 'Dec 08')
results_path = os.path.join('Data/WEAP Results', 'Results')
create_folder(results_path)
for scenario, files in files_dict.items():
    scenario_path = os.path.join(results_path, scenario)
    create_folder(scenario_path)
    for folder, file in files.items():
        folder_path = os.path.join(scenario_path, folder)
        create_folder(folder_path)
        for i, f in enumerate(file):
            dst = os.path.join(folder_path, f'level_{i+1}')
            create_folder(dst)
            copyfile(os.path.join(raw_data, f), os.path.join(dst, 'results.xlsm'))

In [46]:
# scenarios = ['Reference', 'Improve AG eff', 'New Resources', 'Reduce NRW']
scenarios = ['Reference']
processed_data = os.path.join('Data/WEAP Results', 'Processed data')
for scenario in scenarios:
    scenario_folder = os.path.join(results_path, scenario)
    create_folder(os.path.join(processed_data, scenario))
    for sub_scenario in os.listdir(scenario_folder):
        sub_scenario_folder = os.path.join(scenario_folder, sub_scenario)
        create_folder(os.path.join(processed_data, scenario, sub_scenario))
        for level in os.listdir(sub_scenario_folder):
            output_folder = os.path.join(processed_data, scenario, sub_scenario, level)
            create_folder(output_folder)
            file = os.path.join(sub_scenario_folder, level, 'results.xlsm')
            data = pd.ExcelFile(file)

            sheet_names = {'Supply Requirement_AG': 'Agriculture', 
                           'Supply Requirement_Muni': 'Municipality', 
                           'Supply Requirement_Ind': 'Industry'}
            required_demand = get_demand_data(sheet_names, data, demand, 'point', '^ {}')
            required_demand['elevation_m'] = required_demand.point.map(demand.groupby('point')['elevation_m'].mean())

            demand['type'] = demand.point.map(required_demand.groupby('point').agg({'type': 'first'}).type)

            sheet_names = {'Supply Delivered_AG': 'Agriculture', 
                           'Supply Delivered_Muni': 'Municipality', 
                           'Supply Delivered_Ind': 'Industry'}
            delivered_demand = get_demand_data(sheet_names, data, demand, 'point', '^ {}')
            delivered_demand['elevation_m'] = delivered_demand.point.map(demand.groupby('point')['elevation_m'].mean())

            demand.loc[demand['type'].isna(), 'type'] = demand.loc[demand['type'].isna(), 'point'].map(delivered_demand.groupby('point').agg({'type': 'first'}).type)

            sheet_names = {'RedDead': 'Desalination'}
            red_dead = get_demand_data(sheet_names, data, supply, 'point', '{}')
            red_dead = red_dead.append(get_demand_data(sheet_names, data, demand, 'point', '{}'))
            red_dead['point'] = 'RedDead'
            sheet_names = {'Aqaba Desal': 'Desalination'}
            aqaba_desal = get_demand_data(sheet_names, data, demand, 'point', '{}')
            aqaba_desal['point'] = 'Aqaba Desal'
            desalination = red_dead.append(aqaba_desal)
            desalination['value'] = abs(desalination['value'])
            desalination['variable'] = desalination.variable.str.strip()
            
            sheet_names = {'GW Pumping': 'Groundwater supply'}
            gw_demand =  get_demand_data(sheet_names, data, groundwater, 'point', '^ {}')
            gw_demand['wtd_m'] = gw_demand.point.map(groundwater.groupby('point')['wtd_m'].mean())

            sheet_names = {'Groundwater': 'Thickness'}
            gw_thickness =  get_demand_data(sheet_names, data, supply, 'point', '^ {}')
            gw_thickness.rename(columns={'value': 'thickness', 'units': 'thickness_units'}, inplace=True)
            gw_thickness.drop(columns='type', inplace=True)

            for point in gw_thickness.point.unique():
                _filter = (gw_thickness.point==point)
                init_thickness = gw_thickness.loc[_filter].iloc[0].thickness
                gw_thickness.loc[_filter, 'wtd'] = init_thickness - gw_thickness.loc[_filter, 'thickness'] + \
                                                   float(groundwater.loc[groundwater.point==point,'wtd_m'])
            gw_demand = gw_demand.merge(gw_thickness, on=['Year','Month','point','variable'])

            sheet_names = {'Wadis': 'River/pipeline supply'}
            surface_water =  get_demand_data(sheet_names, data, supply, 'point', '{}')

            sheet_names = {'WWTP Inflow': 'WWTP'}
            wwtp_inflow =  get_demand_data(sheet_names, data, supply, 'point', '^ {}')

            pl_flow = pd.DataFrame()

            for sheet_name in ['Pipelines', 'PumpStations']:
                pl_data = data.parse(sheet_name, skiprows=5)
                pl_data.rename(columns={'$Columns = Year': 'Year', ' Timestep': 'Month'}, inplace=True)
                pl_data.columns = pl_data.columns.str.replace('"', '')
                pl_data.columns = pl_data.columns.str.replace('  ', ' ')
                pl_data.columns = pl_data.columns.str.strip()
                if sheet_name == 'Pipelines':
                    pl_data["PL_ZaraMain2SZ09SZ06 0 \ Reach[Cubic Meter]"] = pl_data["PL_ZaraMain2SZ09SZ06 0 \ Headflow[Cubic Meter]"]

                df_pl_temp = extract_weap(pl_data, df_pl, 'pipeline', ['Year','Month'], '^{} [0-9]')
                df_pl_temp['units'] = [extract(s, '\[', '\]') for s in df_pl_temp.variable]
                df_pl_temp['variable'] = [(i).split('[')[0].strip() for i in df_pl_temp.variable]

                pl_flow = pl_flow.append(df_pl_temp, sort=False)

            
            pl_flow = pl_flow.loc[~pl_flow.variable.str.contains('FR')]

            pl_flow['point'] = np.nan
            _vec = ~pl_flow.variable.isin(['Headflow','Reach'])
            pl_flow.loc[_vec,'point'] = pl_flow.loc[_vec,'variable']

            _df = pl_flow.loc[(pl_flow.Year==2020) & (pl_flow.Month==1)].groupby('pipeline').count()
            idx = _df.loc[_df.point<3].index
            _df = pl_flow.loc[pl_flow.pipeline.isin(idx)].copy()
               

            for pipeline in _df.pipeline.unique():
                for row in df_pl.loc[df_pl.pipeline==pipeline].iterrows():
                    if (row[1].point not in _df.loc[(_df.pipeline==pipeline), 'variable'].unique()) & (row[1].type=='Diversion Outflow'):
                        pl_flow.loc[(pl_flow.pipeline==pipeline) & (pl_flow.variable=='Headflow'), 'point'] = row[1].point
                    elif (row[1].point not in _df.loc[(_df.pipeline==pipeline), 'variable'].unique()) & (row[1].type=='Tributary inflow'):
                            pl_flow.loc[(pl_flow.pipeline==pipeline) & (pl_flow.variable=='Reach'), 'point'] = row[1].point

            
            _df = df_pl[['pipeline','point','index']].groupby(['pipeline','point']).mean()
            # _df = _df.reset_index().set_index('point')
            _pl_flow = pl_flow.set_index(['pipeline','point'])
            _index = _pl_flow.index.map(_df['index'].to_dict())
            pl_flow['pl_index'] = [(i + 0.5) if i%1==0.5 else (i) for i in _index]
            pl_flow['elevation'] = pl_flow.point.map(df_pl.groupby('point')['elevation_m'].mean().to_dict())
            pl_flow['segment_length'] = pl_flow.pl_index.map(df_pl.groupby('index')['segment_length_m'].mean().to_dict())
            pl_flow['pipeline_length'] = pl_flow.pipeline.map(df_pl.groupby('pipeline')['pl_length_m'].mean().to_dict())

            pl_flow.dropna(subset=['point'], inplace=True)
            

            enumerate_segments()
            get_elevation_delta()

            pl_flow.loc[(pl_flow.variable=='Reach') & (pl_flow.elevation_delta!=0), 'point'] = np.nan
            pl_flow.dropna(subset=['point'], inplace=True)
            

            enumerate_segments()
            get_elevation_delta()

            _point = supply.loc[supply['type']=='River/pipeline supply'].point.unique()
#             _pipe = pl_flow.loc[pl_flow['point'].isin(_point)].pipeline.unique()
            _pipe = pl_flow.pipeline.unique()
            pl_flow['water_use'] = 0
            for pipe in _pipe:
#                 n = pl_flow.loc[(pl_flow['pipeline']==pipe) & pl_flow['point'].isin(_point)].n.unique()
                n = pl_flow.loc[(pl_flow['pipeline']==pipe)].n.unique()
                for _n in range(1,n.max()+1):
                    
                    value2 = np.array(pl_flow.loc[(pl_flow['pipeline']==pipe) & (pl_flow['n']==_n), 'value'])
                    value1 = np.array(pl_flow.loc[(pl_flow['pipeline']==pipe) & (pl_flow['n']==(_n-1)), 'value'])
                    pl_flow.loc[(pl_flow['pipeline']==pipe) & (pl_flow['n']==_n), 'water_use'] = abs(value1 - value2)
                
                if pl_flow.loc[(pl_flow['pipeline']==pipe)].water_use.sum() == 0:
                    pl_flow.loc[(pl_flow['pipeline']==pipe), 'water_use'] = \
                                                    pl_flow.loc[(pl_flow['pipeline']==pipe), 'value'].mean()/\
                                                    pl_flow.loc[(pl_flow['pipeline']==pipe), 'value'].count()
            
            
            pl_flow['type'] = np.nan
            pl_flow.loc[pl_flow['point'].isin(_point), 'type'] = 'River/pipeline supply'
            
            desalination.to_csv(os.path.join(output_folder, 'Desalination.csv'), index=False)
            wwtp_inflow.to_csv(os.path.join(output_folder, 'WWTP_inflow.csv'), index=False)
            surface_water.to_csv(os.path.join(output_folder, 'Surface_water_supply.csv'), index=False)
            gw_demand.to_csv(os.path.join(output_folder, 'Groundwater_supply.csv'), index=False)
            delivered_demand.to_csv(os.path.join(output_folder, 'Delivered_demand.csv'), index=False)
            required_demand.to_csv(os.path.join(output_folder, 'Required_demand.csv'), index=False)
            pl_flow.to_csv(os.path.join(output_folder, 'Pipelines_flow.csv'), index=False)
            

In [76]:
def get_demand_data(sheet_names, data, spatial_data, variable, regex):
    merged_data = pd.DataFrame()
    for sheet_name, value in sheet_names.items():
        data_temp = data.parse(sheet_name, skiprows=5)
        data_temp.columns = data_temp.columns.str.replace('"', '')
        merged_data_temp = extract_weap(data_temp, spatial_data, variable, ['Year'], regex)
        merged_data_temp['units'] = 'kg'
        merged_data_temp['variable'] = [i.split('[')[0] for i in merged_data_temp.variable]
        merged_data_temp['type'] = value
        merged_data_temp.loc[merged_data_temp['type']=='variable', 'type'] = merged_data_temp.loc[merged_data_temp['type']=='variable', 'variable']

        merged_data = merged_data.append(merged_data_temp, sort=False)
        
    return merged_data

In [117]:
def extract_weap(results, spacial_data, points, melt_on, regex):
    df = pd.DataFrame()
    for key, row in spacial_data.groupby(points):
        df_temp = results.filter(regex=regex.format(key.replace('Agriculture', 'Agri').replace(' ',''))).copy()
        df_temp.columns = [i.split('\\')[-1] for i in df_temp.columns]
        df_temp[points] = key
        for i in melt_on:
            df_temp[i] = results[i]
        melt_vars = melt_on.copy()
        melt_vars.append(points)
        df = df.append(df_temp.melt(id_vars=melt_vars), sort=False)
    return df.reset_index(drop=True)

In [118]:
file = r"Data\WEAP Results\Crop Production.xlsx"
data = pd.ExcelFile(file)
sheet_names = {'WEAP Export': 'Crop production'}
crop_production =  get_demand_data(sheet_names, data, demand, 'point', '^{}')

In [122]:
import plotly.express as px

In [129]:
df = crop_production.loc[crop_production.point.str.contains('AQ')]
df = df.groupby(['Year', 'variable']).sum().reset_index()
fig = px.bar(df, x='Year', y='value', color='variable')
fig.show()

In [112]:
demand.point.unique()

array(['AJ_Agriculture', 'AJ_ROU Ajloun', 'AMMAN_Industry',
       'AM_Agriculture', 'AM_Al Jiza', 'AM_Mwaqqar', 'AM_SZ01', 'AM_SZ02',
       'AM_SZ03', 'AM_SZ04', 'AM_SZ05', 'AM_SZ06', 'AM_SZ07', 'AM_SZ08',
       'AM_SZ09', 'AM_UmBasateyn', 'AQABA_Ind_Fertilizer',
       'AQABA_Industry', 'AQ_Agri', 'AQ_Al Quwayra', 'AQ_QAqaba',
       'BALQA_Industry', 'BQ_AgriHL', 'BQ_AgriJV', 'BQ_Ain Al Basha',
       'BQ_Deir Alla', 'BQ_Mahis Fuhais', 'BQ_ROU_Salt', 'BQ_ShJanoubia',
       'IRBID_Industry', 'IR_AgriHL', 'IR_AgriJV', 'IR_Al Aghwar Sh',
       'IR_Bani Kinana', 'IR_Bani Obeid', 'IR_Koura', 'IR_RC_C City',
       'IR_RC_KAP', 'IR_ROU_Irbid', 'IR_Ramtha', 'JA_Agriculture',
       'JA_ROU Jerash', 'JERASH_Industry', 'KARAK_Ind_Potash',
       'KARAK_Industry', 'KA_Agri HL', 'KA_AgriJV', 'KA_Al MazarJan',
       'KA_Al Qasr', 'KA_Al Qatrana', 'KA_AlAg Janoub', 'KA_Ayy',
       'KA_Fqou', 'KA_Ghor Al Mazraa', 'KA_Ind_OilShale',
       'KA_Q  Al Karak', 'MAAN_Industry', 'MADABA_Industry'

In [33]:
folder = r'../Jordan dashboard/spatial_data/'
demand.to_file(f'{folder}Demand_points.gpkg', driver='GPKG')
supply.to_file(f'{folder}Supply_points.gpkg', driver='GPKG')
df_pl.to_file(f'{folder}Pipelines.gpkg', driver='GPKG')